# Police stations: Police_Stations_-_Map.csv
# Libraries: Libraries_-Locations_Hours_and_Contact_Information.csv
# Schools: Chicago Public Schools - School Loactions SY1011.geojson
# Todo: for each police report, find distance to nearest station, to nearest library, and to nearest school

In [42]:
import csv
import time
from collections import namedtuple
from math import sqrt
import geopandas as gpd
import pandas as pd
import shapely
from shapely.ops import nearest_points
import numpy as np
from scipy import ndimage
from scipy.spatial import cKDTree  

import matplotlib
from matplotlib.pyplot import figure
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from matplotlib.colors import ListedColormap
from matplotlib.ticker import MaxNLocator

In [44]:
DTN_Args = namedtuple("DTN_Args", ['geom_union', 'df1', 'df2', 'geom1_col', 'geom2_col', 'src_column'])

In [40]:
#def distance_to_nearest(row, geom_union, df1, df2, geom1_col='geometry', geom2_col='geometry', src_column=None):
def distance_to_nearest(row, dtn_args):
     """Find the nearest point and return the corresponding value from specified column."""
     #distances = []
     df1 = dtn_args.df1
     df2 = dtn_args.df2
     geom1_col = dtn_args.geom1_col
     geom2_col = dtn_args.geom2_col
     geom_union = dtn_args.geom_union
     src_column = dtn_args.src_column if dtn_args.src_column is not None else None
     # Find the geometry that is closest
     nearest = df2[geom2_col] == nearest_points(row[geom1_col], geom_union)[1]
     # Get the corresponding value from df2 (matching is based on the geometry)
     nearest_id = df2[nearest][src_column].get_values()[0]
     start_point: shapely.geometry.Point = row['geometry']
     end_point_series: pd.Series = df2[nearest]['geometry']
     end_point: shapely.geometry.Point = end_point_series.iloc[0]
     dist_to_point = start_point.distance(end_point)
     #distances.append(dist_to_point)
     return nearest_id, dist_to_point

In [3]:
crimes = pd.read_pickle("crimes.pkl")
neighborhoods = pd.read_pickle("neighborhoods.pkl")
with_incomes = pd.read_pickle("with_incomes.pkl")

In [4]:
crimes2 = None
crimes2 = pd.read_pickle('crimes2.pkl')

In [5]:
crimes2.head()

,report_id,date,primary type,category,location description,arrest,community area,geometry
0,8265244,09/15/2011,BURGLARY,property,RESIDENCE,true,false,POINT (-87.771918163 41.95396528)
1,4991465,01/01/2001,DECEPTIVE PRACTICE,other,"SCHOOL, PRIVATE, GROUNDS",true,false,POINT (-87.660604177 42.001678618)
2,5147738,08/01/2001,OFFENSE INVOLVING CHILDREN,person,RESIDENCE,true,false,POINT (-87.66343916300001 41.802219519)
3,5281401,02/26/2001,OTHER OFFENSE,other,HOSPITAL BUILDING/GROUNDS,false,false,POINT (-87.61391851099999 41.802238398)
4,5622147,12/01/2001,OFFENSE INVOLVING CHILDREN,person,RESIDENCE,false,true,POINT (-87.719768573 41.931044901)


In [6]:
crimes = crimes.astype({'id': str})
type(crimes.iloc[0]['id'])

str

In [16]:
schools = None
schools = pd.read_pickle("schools.pkl")

In [ ]:
# Convert GPS to UTM. Chicago is in UTM zone 16T
myProj = Proj("+proj=utm +zone=16T, +south +ellps=WGS84 +datum=WGS84 +units=m +no_defs")
lon, lat = myProj(df['Meters East'].values, df['Meters South'].values, inverse=True)
UTMx, UTMy = myProj(lon, lat)

In [23]:
# Read in a subset of the crimes dataset for testing the nearest neighbor methods
crimes_extract = pd.read_csv("parallel/xaa")
crimes_extract.head()

SyntaxError: can't assign to operator (<ipython-input-23-65ac82290993>, line 2)

In [15]:
schools.to_pickle("schools.pkl")

In [31]:
schools.set_index("UNIT_ID")
schools['UNIT_ID'] = schools.index
schools.head

<bound method NDFrame.head of                                              geometry  \
UNIT_ID                                                 
4610             POINT (-87.5748539059 41.7483660139)   
5180          POINT (-87.57284397069999 41.768650177)   
5300              POINT (-87.5829920307 41.768594283)   
5440          POINT (-87.60800008530001 41.740365734)   
5610          POINT (-87.60015764629999 41.764223475)   
5670         POINT (-87.63887832290001 41.7353396836)   
5880         POINT (-87.55282162479999 41.7315965518)   
6100         POINT (-87.54289106020001 41.7438333435)   
6350             POINT (-87.5753168646 41.7268027887)   
6900         POINT (-87.60113360690001 41.7400743261)   
1135         POINT (-87.66366659329999 41.9028783879)   
2123          POINT (-87.66002579629999 41.918307417)   
2123         POINT (-87.68663350950001 41.8830789626)   
1105             POINT (-87.7080245074 41.8584605337)   
6320         POINT (-87.6480500744 41.85485633010001)   
6

In [17]:
# Create unary_union object from the schools dataset
schools_unary_union = schools.unary_union

In [18]:
#print(schools['geometry'] == nearest_points(crimes.iloc[2]['geometry'], schools_unary_union))
print(crimes.iloc[2]['geometry'].distance(schools.iloc[100]['geometry']))
#print(crimes.iloc[2]['geometry'])

0.09386390354079342


In [35]:
# Locate the nearest school to each reported crime's location
start_time = time.time
unpackdf = pd.DataFrame(crimes2.apply(distance_to_nearest, axis=1,
                                      args=(
                                          geom_union=schools_unary_union,
                                          df1=crimes2,
                                          df2=schools,
                                          src_column='UNIT_ID',
                                      )).tolist(),
                        columns = ['nearest_school_id',
                                   'nearest_school_distance'],
                        index=crimes2.index)
crimes2=pd.concat([unpackdf, crimes2], axis=1)
end_time = time.time
print("That took {0} seconds".format(end_time - start_time))

SyntaxError: invalid syntax (<ipython-input-35-fdf2658532c9>, line 5)

In [73]:
crimes2.head()

,report_id,date,primary type,category,location description,arrest,community area,geometry
0,8265244,09/15/2011,BURGLARY,property,RESIDENCE,true,false,POINT (-87.771918163 41.95396528)
1,4991465,01/01/2001,DECEPTIVE PRACTICE,other,"SCHOOL, PRIVATE, GROUNDS",true,false,POINT (-87.660604177 42.001678618)
2,5147738,08/01/2001,OFFENSE INVOLVING CHILDREN,person,RESIDENCE,true,false,POINT (-87.66343916300001 41.802219519)
3,5281401,02/26/2001,OTHER OFFENSE,other,HOSPITAL BUILDING/GROUNDS,false,false,POINT (-87.61391851099999 41.802238398)
4,5622147,12/01/2001,OFFENSE INVOLVING CHILDREN,person,RESIDENCE,false,true,POINT (-87.719768573 41.931044901)


In [88]:
crimes_extract = crimes2.iloc[0]
crimes2['category'].value_counts()

property    350564
person      179753
other       101029
vice         89991
Name: category, dtype: int64

In [60]:
dtn_args = DTN_Args(geom_union=schools_unary_union,
                    df1=crimes_extract,
                    df2=schools,
                    geom1_col='geometry',
                    geom2_col='geometry',
                    src_column="UNIT_ID"
                   )
#schools[dtn_args.geom2_col] == nearest_points(crimes_extract.iloc[0][dtn_args.geom1_col], dtn_args.geom_union)[1]
schools[dtn_args.geom2_col] == nearest_points(crimes_extract['geometry'], dtn_args.geom_union)[1]

UNIT_ID
4610    False
5180    False
5300    False
5440    False
5610    False
5670    False
5880    False
6100    False
6350    False
6900    False
1135    False
2123    False
2123    False
1105    False
6320    False
6450    False
6570    False
7190    False
7610    False
5640    False
6490    False
4230    False
4310    False
4430    False
4430    False
4710    False
5430    False
5970    False
7720    False
4330    False
        ...  
3000    False
3240    False
3410    False
3660    False
3910    False
3950    False
4110    False
4640    False
5350    False
5450    False
6210    False
6340    False
6380    False
6590    False
6610    False
6730    False
6740    False
6920    False
7840    False
8024    False
2070    False
2740    False
2890    False
2930    False
3100    False
3750    False
3960    False
3970    False
4020    False
4210    False
Name: geometry, Length: 688, dtype: bool

In [71]:
df = pd.DataFrame({'A':[1,2,3],
                   'B':[4,5,6]})

print (df)

def myfunction(B, A):
    #some staff  
    result = B + A 
    # do something here to get the result
    return result

df['C'] = df.apply(lambda x: myfunction(x.B, x.A), axis=1)
print (df)

   A  B
0  1  4
1  2  5
2  3  6
   A  B  C
0  1  4  5
1  2  5  7
2  3  6  9


In [81]:
#np.sum(crimes_extract['report_id'], np.array([100])

str

In [84]:
crimes_extract.apply(lambda x: x['arrest'] && x['community area'])

SyntaxError: invalid syntax (<ipython-input-84-ed6fd95bb485>, line 1)

In [72]:
start_time = time.time
unpackdf = pd.DataFrame(crimes_extract.apply(lambda: distance_to_nearest(dtn_args=dtn_args), axis=1).to_list(),
                        columns = ['nearest_school_id', 'nearest_school_distance'],
                        index = crimes_extract.index)

TypeError: <lambda>() got an unexpected keyword argument 'axis'

In [67]:
# Locate the nearest school to each reported crime's location

start_time = time.time
#unpackdf = pd.DataFrame(crimes_extract.apply(distance_to_nearest, dtn_args, axis=1).to_list(),
unpackdf = pd.DataFrame(crimes_extract.apply(distance_to_nearest, args=(dtn_args), axis=1).to_list(),
                                             columns = ['nearest_school_id', 'nearest_school_distance'],
                                             index = crimes_extract.index)
#                        columns = ['nearest_school_id', 'nearest_school_distance'],
#                        index = crimes_extract.index)
#                        columns = ['nearest_school_id',
#                                   'nearest_school_distance'
#                                  ],
#                        index=crimes_extract.index)

crimes_extract=pd.concat([unpackdf, crimes_extract], axis=1)
end_time = time.time
print("That took {0} seconds".format(end_time - start_time))

TypeError: distance_to_nearest() got an unexpected keyword argument 'axis'

In [153]:
test_start['nearest_id'] = test_start.apply(nearest_id, geom_union=unary_union, df1=test_start, df2=test_end, src_column='id', axis=1)
test_start

Nearest IDs: ['epoint4']
Distances: [<shapely.geometry.point.Point object at 0xb2371d7f0>]
Nearest IDs: ['epoint5']
Distances: [<shapely.geometry.point.Point object at 0xb2371d828>]
Nearest IDs: ['epoint4']
Distances: [<shapely.geometry.point.Point object at 0xb2371d7f0>]


,id,geometry,nearest_id
spoint1,spoint1,POINT (1 0),epoint4
spoint2,spoint2,POINT (5 0),epoint5
spoint3,spoint3,POINT (3 5),epoint4


In [142]:
test_start['nearest_id'] = test_start.apply(nearest_id, geom_union=unary_union, df1=test_start, df2=test_end, src_column='id', axis=1)
distances = []
distance_indecies = []
for row in test_start.itertuples():
    start_point = getattr(row, 'geometry')
    end_point = test_end.loc[getattr(row, 'nearest_id')]['geometry']
    distances.append(start_point.distance(end_point))
    distance_indecies.append(row.id)
new_column = pd.Series(distances, index=distance_indecies)
test_start['nearest_distance'] = new_column
print(new_column)
test_start

spoint1    4.123106
spoint2    3.000000
spoint3    1.414214
dtype: float64


,id,geometry,nearest_id,nearest_distance
spoint1,spoint1,POINT (1 0),epoint4,4.123106
spoint2,spoint2,POINT (5 0),epoint5,3.000000
spoint3,spoint3,POINT (3 5),epoint4,1.414214


In [32]:
(test_start['nearest_id'], test_start['nearest_point']) = test_start.apply(nearest,
                                                                         geom_union=unary_union,
                                                                         df1=test_start,
                                                                         df2=test_end,
                                                                         src_column='id', axis=1)

Nearest(<class 'geopandas.geoseries.GeoSeries'>): epoint1    False
epoint2    False
epoint3    False
epoint4     True
epoint5    False
Name: geometry, dtype: bool
POINT (2 4)
Nearest(<class 'geopandas.geoseries.GeoSeries'>): epoint1    False
epoint2    False
epoint3    False
epoint4    False
epoint5     True
Name: geometry, dtype: bool
POINT (8 0)
Nearest(<class 'geopandas.geoseries.GeoSeries'>): epoint1    False
epoint2    False
epoint3    False
epoint4     True
epoint5    False
Name: geometry, dtype: bool
POINT (2 4)


ValueError: too many values to unpack (expected 2)

In [146]:
df = pd.DataFrame([[4, 9],] * 3, columns=['A', 'B'])
df
df['foo'], df['bar'] = df.apply(lambda x: pd.Series([1, 2], index=['foo', 'bar']), axis=1)
df

,A,B,foo,bar
0,4,9,foo,bar
1,4,9,foo,bar
2,4,9,foo,bar


In [151]:
start_uu = test_start.unary_union

I'm working on a personal-learning project where I'm trying to figure out what the nearest point from dataframe B is to each point in in dataframe A. I've got that bit working, but it only tells me what the nearest point is. I also want to know how far away the nearest point is, but I'm not figuring it out.

I can find the distance manually one point at a time, but my efforts to do so for the whole list have failed thus far.

Simplified examples of the two dataframes are:

    test_start = gpd.GeoDataFrame([{"id": "spoint1", "geometry": shapely.geometry.Point(1, 0)},
                                   {"id": "spoint2", "geometry": shapely.geometry.Point(5, 0)}])

    test_end = gpd.GeoDataFrame([{"id": "epoint1", "geometry": shapely.geometry.Point(-3, 3)},
                                 {"id": "epoint2", "geometry": shapely.geometry.Point(-1, 4)},
                                 {"id": "epoint3", "geometry": shapely.geometry.Point(5, 4)},
                                 {"id": "epoint4", "geometry": shapely.geometry.Point(2, 4)},
                                 {"id": "epoint5", "geometry": shapely.geometry.Point(8, 0)}
                                ])

The nearest_point method I've cargo-culted from [this tutorial](https://automating-gis-processes.github.io/2017/lessons/L3/nearest-neighbour.html) is:

    def nearest(row, geom_union, df1, df2, geom1_col='geometry', geom2_col='geometry', src_column=None):
        """Find the nearest point and return the corresponding value from specified column."""
        # Find the geometry that is closest
        nearest = df2[geom2_col] == nearest_points(row[geom1_col], geom_union)[1]
        #nearest = df2[geom2_col] == nearest_points(df1, df2)[1]
        # Get the corresponding value from df2 (matching is based on the geometry)
        value = df2[nearest][src_column].get_values()[0]
        return value

which is called from:

    test_start['nearest_id'] = test_start.apply(nearest,
                                                geom_union=unary_union,
                                                df1=test_start,
                                                df2=test_end,
                                                src_column='id', axis=1)

What I want is another column in test_start that contains the distance between the start and end points. As mentioned above, I can do this manually with something like:

    print(test_start["geometry"][0].distance(test_end["geometry"][3]))

but I can't figure out how to do it for the whole test_start dataframe.

This is a very simplified data set. The real starting point data set has over 7 million rows and the end set has around 800, so efficiency may become a factor if the brute-force method takes half a day or more.

Using apply() seemed like it would be a path to get to what I wanted, but I can't figure out how to write it so the system knows which points are being compared.

Somewhat closer. This code gets past the indexing, etc. problems, but appears to select just one end point as being nearest to all of the start points.

     def distance_to_nearest(row, geom_union, df1, df2, geom1_col='geometry', geom2_col='geometry', src_column=None):
         """Find the nearest point and return the corresponding value from specified column."""
         distances = []
         # Find the geometry that is closest
         nearest = df2[geom2_col] == nearest_points(row[geom1_col], geom_union)[1]
         # Get the corresponding value from df2 (matching is based on the geometry)
         nearest_id = df2[nearest][src_column].get_values()[0]
         start_point = row['geometry']
         end_point_series = df2[nearest]['geometry']
         end_point = end_point_series.iloc[0]
         dist_to_point = start_point.distance(end_point)
         distances.append(dist_to_point)
         return nearest_id, dist_to_point